In [3]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import random
import os

from dev.preprocessing import faceMesh_video
from dev.preprocessing import handPose_video
from dev.preprocessing import points_to_displacement

In [5]:
# 오래걸림!!! 주의하셈 1분 넘게 걸린다.
# Long time to run!! Caution.
with open('../dataset/processed/KSL_handpose_and_facemesh.p', 'rb') as handle:
    dataset = pickle.load(handle)

In [6]:
for i in range(1540):
    index = dataset[i][1]-1
    vector = [0 for _ in range(77)]
    vector[index] = 1
    dataset[i][1] = vector
    

In [7]:
# print(dataset[0])
print(len(dataset))
TRAIN_SPLIT = 1232
# 0.8 * 1540 = 1232 개

1540


In [8]:
print(len(dataset[0][0][0])) # 478 + 21 * 2

520


In [16]:

random.shuffle(dataset)

train_data = dataset[:1232]
test_data = dataset[1232:]

x_train = []
y_train = []
x_test = []
y_test = []


for data in train_data:
    x_train.append(np.array(data[0], dtype=np.float32))
    y_train.append(np.array(data[1], dtype=np.float32))

for data in test_data:
    x_test.append(np.array(data[0], dtype=np.float32))
    y_test.append(np.array(data[1], dtype=np.float32))

x_train = np.array(x_train)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (100, 520) + inhomogeneous part.

In [13]:
print(len(train_data))
print(len(train_data[1]))
print(len(train_data[1][0]))
print(len(train_data[1][0][1]))

1232
2
100
520


In [ ]:
shape_points = (100, 520, 3)

model = keras.models.Sequential()
model.add(keras.Input(shape=shape_points, dtype='float32', name='points_input'))
model.add(keras.layers.GRU(32, input_shape=shape_points[-2:]))
model.add(keras.layers.Dense(256), activation='relu')
model.add(keras.layers.Dense(77), activation='softmax')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:

history = model.fit(x_train, y_train, epochs=10, batch_size=256, validation_data=(x_test, y_test))
